# Benchmarking Collaborative Filtering Recommendation Algorithms

The benchmarking applies to collaborative filtering algorithms available in Microsoft/Recommenders repository like Spark ALS, Surprise SVD, Microsoft SAR, etc.

## Experimentation setup:
* Objective
  * To compare how each collaborative filtering algorithm perform in recommending list of items.
* Datasets
  * Movielens 100K.
  * Movielens 1M.
  * Movielens 10M.
  * Movielens 20M.
* Data split
  * The data is split into train and test sets.
  * The split ratios are 75-25 for train and test datasets.
  * The splitting is random. 
* Model training
  * A recommendation model is trained by using each of the collaborative filtering algorithms. 
  * It is known that exhaustive search of the hyper parameter space is cubersome. Instead, empirical parameter values reported in the literature that generated optimal results are used.
* Evaluation metrics
  * Precision@k.
  * Recall@k.
  * Normalized discounted cumulative gain@k (NDCG@k).
  * Mean-average-precision (MAP). 
  * In the evaluation metrics above, k = 10. 

## 0 Global settings

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("../../")
import os
import numpy as np
import pandas as pd
from zipfile import ZipFile
import papermill as pm
import time
import itertools

import pyspark
from pyspark.ml.recommendation import ALS
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, FloatType, IntegerType

import surprise

from reco_utils.dataset.url_utils import maybe_download
from reco_utils.dataset.movielens import load_spark_df, load_pandas_df
from reco_utils.dataset.python_splitters import python_random_split, python_chrono_split, python_stratified_split
from reco_utils.dataset.spark_splitters import spark_chrono_split
from reco_utils.recommender.sar.sar_singlenode import SARSingleNodeReference
from reco_utils.evaluation.spark_evaluation import SparkRankingEvaluation
from reco_utils.evaluation.python_evaluation import (
    map_at_k,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    rsquared,
    exp_var
)
from reco_utils.evaluation.parameter_sweep import generate_param_grid

print("System version: {}".format(sys.version))
print("Spark version: {}".format(pyspark.__version__))

System version: 3.6.0 | packaged by conda-forge | (default, Feb  9 2017, 14:36:55) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]
Spark version: 2.3.1


In [2]:
%env PYSPARK_PYTHON=/anaconda/envs/recommender/bin/python3
%env PYSPARK_DRIVER_PYTHON=/anaconda/envs/recommender/bin/python3

env: PYSPARK_PYTHON=/anaconda/envs/recommender/bin/python3
env: PYSPARK_DRIVER_PYTHON=/anaconda/envs/recommender/bin/python3


In [3]:
# Configure Spark
spark = SparkSession \
    .builder \
    .appName("ALS pySpark") \
    .master("local[*]") \
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.cores", "32")\
    .config("spark.executor.memory", "8g")\
    .config("spark.memory.fraction", "0.9")\
    .config("spark.memory.stageFraction", "0.3")\
    .config("spark.executor.instances", 1)\
    .config("spark.executor.heartbeatInterval", "36000s")\
    .config("spark.network.timeout", "10000000s")\
    .config("spark.driver.maxResultSize", "50g")\
    .getOrCreate()

In [4]:
# top k items to recommend
TOP_K = 10

# Select Movielens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZES = ['100k', '1m', '10m', '20m']

# Metric for selection
SELECTION_METRIC = "recall"

## 1 Prepare data

In [5]:
# Set data schema
headers = {
    "col_user": "UserId",
    "col_item": "MovieId",
    "col_timestamp": "Timestamp"
}

In [6]:
# Download data
MOVIELENS_DATA_SIZE = '100k'
data = load_pandas_df(size=MOVIELENS_DATA_SIZE)

In [7]:
# Split data w.r.t the experimentation protocol.
df_train, df_test = python_random_split(data, ratio=0.75, seed=123)

## 2 Train model

CF algorithms available in the repo are comparatively studied. They are Spark ALS, SAR, and Surprise SVD.

In [8]:
cf_algorithms = ["als", "sar", "svd"]

In [9]:
def recommender(
    spark,
    data_train,
    data_test,
    algo="als",
    col_user="UserId",
    col_item="MovieId",
    col_rating="Rating",
    col_timestamp="Timestamp",
    **params
):
    if algo == "als":  
        als = ALS(
            implicitPrefs=False,
            coldStartStrategy='drop',
            userCol=col_user,
            itemCol=col_item,
            ratingCol=col_rating,
            nonnegative=False,
            **params
        )

        dfs_train = spark.createDataFrame(df_train)
        dfs_test = spark.createDataFrame(df_test)

        time_start = time.time()
        model = als.fit(dfs_train)
        time_train = time.time() - time_start

        time_start = time.time()
        
#         dfs_rec = model.recommendForUserSubset(dfs_test, TOP_K)
        item_count = len(set(df_train[col_item].unique()))

        dfs_rec = model.recommendForAllUsers(item_count)
        dfs_pred = dfs_rec.select(col_user, F.explode('recommendations').alias('r')) \
          .select(col_user, 'r.*')
        
        print(dfs_pred.show())
        print(dfs_pred.groupBy(col_user).count().show())
        print(dfs_train.groupBy(col_user).count().show())
        
        # Remove seen items.
        dfs_pred_exclude_train = dfs_pred.alias("pred").join(
            dfs_train.alias("train"),
            (dfs_pred[col_user] == dfs_train[col_user]) & (dfs_pred[col_item] == dfs_train[col_item]),
            how='outer'
        )
        
        dfs_pred_final = dfs_pred_exclude_train.filter(dfs_pred_exclude_train["pred.rating"].isNotNull()) \
            .select('pred.' + col_user, 'pred.' + col_item, 'pred.' + "rating")
        print(dfs_pred_final.show())

        time_test = time.time() - time_start

        df_pred = dfs_pred_final.withColumnRenamed("rating", "prediction").toPandas()
    elif algo == "sar":
        model = SARSingleNodeReference(
            remove_seen=True, 
            time_now=None, 
            timedecay_formula=True, 
            col_rating=col_rating,
            **headers,
            **params
        )
        
        data = data_train.append(data_test)

        time_start = time.time()
        unique_users = data[col_user].unique()
        unique_items = data[col_item].unique()

        enumerate_items_1, enumerate_items_2 = itertools.tee(enumerate(unique_items))
        enumerate_users_1, enumerate_users_2 = itertools.tee(enumerate(unique_users))
        item_map_dict = {x: i for i, x in enumerate_items_1}
        user_map_dict = {x: i for i, x in enumerate_users_1}

        index2user = dict(enumerate_users_2)
        index2item = dict(enumerate_items_2)

        model.set_index(unique_users, unique_items, user_map_dict, item_map_dict, index2user, index2item)

        df_train_sar = df_train.copy()
        model.fit(df_train_sar)
        time_train = time.time() - time_start

        time_start = time.time()
        df_test_sar = df_test.copy()
        top_k = model.recommend_k_items(df_test_sar)

        top_k[col_user] = pd.to_numeric(top_k[col_user])
        top_k[col_item] = pd.to_numeric(top_k[col_item])
        time_test = time.time() - time_start

        df_pred = top_k
    elif algo == "svd":
        df_train_svd = df_train[[col_user, col_item, col_rating]]
        train = surprise.Dataset.load_from_df(df_train_svd, reader=surprise.Reader('ml-100k')).build_full_trainset()

        svd = surprise.SVD(
            verbose=False,
            **params
        )

        time_start = time.time()
        svd.fit(train)
        time_train = time.time() - time_start

        time_start = time.time()
#         predictions = [svd.predict(row.UserId, row.MovieId, row.Rating)
#                for (_, row) in df_test.iterrows()]
#         predictions = pd.DataFrame(predictions)
#         predictions = predictions.rename(index=str, columns={'uid': col_user, 'iid': col_item,
#                                                              'est': 'prediction'})
#         predictions = predictions.drop(['details', 'r_ui'], axis='columns')
        
        # To make sure the predictions include items in the overall dataset.
        preds_lst = []
        for user in df_train[col_user].unique():
            for item in df_train[col_item].unique():
                preds_lst.append([user, item, svd.predict(user, item).est])
                
        all_predictions = pd.DataFrame(data=preds_lst, columns=[col_user, col_item, "prediction"])
        merged = pd.merge(df_train, all_predictions, on=[col_user, col_item], how="outer")
        all_predictions = merged[merged[col_rating].isnull()].drop(col_rating, axis=1)
        
        df_pred = all_predictions
        
        time_test = time.time() - time_start
    else:
        raise ValueError("No algorithm {} found".format(algo))

    return df_pred, time_train, time_test

In [10]:
def compute_metrics(
    data_pred,
    data_true,
    col_user="UserId",
    col_item="MovieId",
    col_rating="Rating",
    col_prediction="prediction",
    k=TOP_K
):
    # Ranking metrics.
    eval_map = map_at_k(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction, 
        k=k
    )
    
    eval_ndcg = ndcg_at_k(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction, 
        k=k
    )
    
    eval_precision = precision_at_k(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction, 
        k=k
    )
    
    eval_recall = recall_at_k(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction, 
        k=k
    )
    
    # Rating metrics.
    eval_rmse = rmse(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction
    )
    
    eval_mae = mae(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction
    )
    
    eval_r2 = rsquared(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction
    )
    
    eval_expvar = exp_var(
        data_true, data_pred, 
        col_user=col_user, col_item=col_item, col_rating=col_rating, col_prediction=col_prediction
    )
    
    df_result = pd.DataFrame(
        {
            "Algo": algo,
            "K": TOP_K,
            "MAP": eval_map,
            "nDCG@k": eval_ndcg,
            "Precision@k": eval_precision,
            "Recall@k": eval_recall,
            "RMSE": eval_rmse,
            "MAE": eval_mae,
            "R2": eval_r2,
            "Explained Variance": eval_expvar,
            "Train time": time_train,
            "Test time": time_test
        }, 
        index=[0]
    )
    
    return df_result

Instead of a time-consuming hyper parameter searching, hyper parameters that are empirically selected to train models for each algorithms. These parameters are determined either by referencing to the literature or empirically.

In [11]:
cf_params = {
    "als": {
        "rank": 10,
        "regParam": 0.05,
        "maxIter": 75,
        "seed": 123
    },
    "sar": {
        "time_decay_coefficient": 30,
        "similarity_type": "jaccard"
    },
    "svd": {
        "random_state": 123,
        "n_factors": 160,
        "n_epochs": 100,
        "lr_all": 0.07,
        "reg_bu": 0.003,
        "reg_bi": 0.003,
        "reg_pu": 0.08,
        "reg_pi": 0.1
    }
}

# http://mymedialite.net/examples/datasets.html

# num_factors=10 num_iter=75 reg=0.05 learn_rate=0.005

# num_factors=160 bias_reg=0.003 reg_u=0.08 reg_i=0.1 learn_rate=0.07 num_iter=100 bold_driver=true

In [12]:
df_results = pd.DataFrame()

for idx, algo in enumerate(cf_algorithms):
    params = cf_params[algo]
    
    df_pred, time_train, time_test = recommender(
        spark,
        df_train,
        df_test,
        algo=algo,
        col_user="UserId",
        col_item="MovieId",
        col_rating="Rating",
        col_timestamp="Timestamp",
        **params
    )

    df_result = compute_metrics(
        df_pred,
        df_test,
        col_user="UserId",
        col_item="MovieId",
        col_rating="Rating",
        col_prediction="prediction",
        k=TOP_K
    )
    
    # Rating metrics do not apply to certain algorithms.
    if algo == "sar":
        df_result[["RMSE", "MAE", "R2", "Explained Variance"]] = np.nan

    df_results = df_results.append(df_result, ignore_index=True)
        
df_results

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 52930)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda/envs/recommender/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/anaconda/envs/recommender/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda/envs/recommender/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/serializers.py", line 685, in read_int
    raise EOFError
EOFError
---

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

...

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

An error occurred while trying to connect to the Java server (127.0.0.1:37317)
Traceback (most recent call last):
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-5e70a4d61438>", line 15, in <module>
    **params
  File "<ipython-input-9-7b8c16388f1f>", line 27, in recommender
    model = als.fit(dfs_train)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/anaconda/envs/recommender/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:37317)